# Using AWS Bedrock

In [ ]:
%pip install --user boto3

# Using OpenAI
Following tutorial from LangChain https://python.langchain.com/v0.2/docs/tutorials/rag/

In [1]:
%pip install --user -qU langchain
%pip install --user -qU langchain_community
%pip install --user -qU langchain_chroma
%pip install --user -qU langchain-openai
%pip install --user -qU langchainhub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [4]:
folder_path = "ca-md"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = UnstructuredMarkdownLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [6]:
text_chunks[0].metadata

{'source': '01-binance.md', 'start_index': 0}

In [7]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())

### Create a retriever

LangChain defines a Retriever interface which wraps an index that can return relevant Documents given a string query.

Function as_retriever offers 3 search type:
- similarity (default)
- mmr
- similarity_score_threshold

In [8]:
retriever = vectorstore.as_retriever()

Using prompt template from LangChain prompt hub
(https://smith.langchain.com/hub/rlm/rag-prompt)

Template: \
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. \
Question: {question} \
Context: {context} \
Answer:

In [10]:
# define format_docs function to join the retrieved text chunks into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("rlm/rag-prompt")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
rag_chain.invoke("How to cancel order?")

'To cancel an order, you can use the cancel_order function in the trade() or on_tradingview_signal() contexts. Additionally, the cancel_order_by_client_order_id function can also be used in the same contexts. Both functions require specific inputs to cancel orders.'

Customizing our own prompt

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "How to cancel order?"})
print(response["answer"])

To cancel an order, you can use the `cancel_order` function with the order object as input. This function can only be used within the `trade()` or `on_tradingview_signal()` functions. Here is an example: `CA.cancel_order(order)`.
